---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    d={'State':[],'RegionName':[]}
    f = open('university_towns.txt', 'r')
    for i in f:
        if '[edit]' in i:
            state=i
            #d['State'].append(i[:i.index('[')])
        elif " (" in i:
            d['RegionName'].append(i[:i.index('(')-1].rstrip())
            d['State'].append(state[:state.index('[')])
        #elif ":" in i:
         #   d['RegionName'].append(i[:i.index(':')].rstrip())
          #  d['State'].append(state[:state.index('[')])
        else:
            d['RegionName'].append(i.rstrip())
            d['State'].append(state[:state.index('[')])
    df=pd.DataFrame(d,columns=['State','RegionName'])
    #df=df[['State','RegionName']]
    return df
    #print([i.rstrip() for i in f if '[edit]' in i])

In [ ]:
#ls=[]
#for name in (get_list_of_university_towns()==get_list_of_university_towns1())['RegionName']:
#    if name==False:
#        #########179
         #########184both misses ':' at end
#print(ls)
#(get_list_of_university_towns()==get_list_of_university_towns1())[(get_list_of_university_towns()==get_list_of_university_towns1())['RegionName']==False]
#get_list_of_university_towns().iloc[[179,184]]

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',header=5)
    df=df.iloc[2:,3:5]
    #df=pd.concat([df.iloc[:,0],df.iloc[:,2]],axis=1)
    df.columns=[['Year','GDP']]
    df=df.iloc[np.where([df['Year']=='2000q1'])[1][0]:]
    #df.reindex([i for i in range(len(df))])
    df.index=[i for i in range(len(df))]
    for i in range(len(df)):
        if (df.iloc[i]['GDP']>df.iloc[i+1]['GDP']) and (df.iloc[i+1]['GDP']>df.iloc[i+2]['GDP']):
            return df.iloc[i]['Year']
    return df

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',header=5)
    df=df.iloc[2:,3:6]
    df=pd.concat([df.iloc[:,0],df.iloc[:,2]],axis=1)
    df.columns=[['Year','GDP']]
    df=df.iloc[np.where([df['Year']=='2000q1'])[1][0]:]
    #df.reindex([i for i in range(len(df))])
    df.index=[i for i in range(len(df))]
    for i in range(len(df)):
        if (df.iloc[i]['GDP']>df.iloc[i+1]['GDP']) and (df.iloc[i+1]['GDP']>df.iloc[i+2]['GDP']) and (df.iloc[i+2]['GDP']<df.iloc[i+3]['GDP']) and (df.iloc[i+3]['GDP']<df.iloc[i+4]['GDP']):
            return df.iloc[i+4]['Year']
    #return df

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel('gdplev.xls',header=5)
    df=df.iloc[2:,3:6]
    df=pd.concat([df.iloc[:,0],df.iloc[:,2]],axis=1)
    df.columns=[['Year','GDP']]
    df=df.iloc[np.where([df['Year']=='2000q1'])[1][0]:]
    #df.reindex([i for i in range(len(df))])
    df.index=[i for i in range(len(df))]
    ls=list()
    for i in range(df[df['Year']==get_recession_start()].index[0],df[df['Year']==get_recession_end()].index[0]+1):
        ls.append(df.iloc[i]['GDP'])
    return df.iloc[np.where(df['GDP']==min(ls))[0][0]]['Year']

In [ ]:
k=2
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df=pd.read_csv('City_Zhvi_AllHomes.csv')
    #print(df.columns)
    df2=df[['State','RegionName']]
    df3=df[['State','RegionName']]
    df2=pd.concat([df2,df.loc[:,'2000-01':]],axis=1)
    #print(df2.iloc[2,0])
    #print(df3.iloc[2,0])
    for i in df2.index:
        df2.iloc[i,0]=states[df2.iloc[i,0]]
        df3.iloc[i,0]=states[df3.iloc[i,0]]
    #df2.set_index(['State',"RegionName"],inplace=True)
    def func():
        global k
        if k<=197:
            temp=df2.iloc[:,k:k+3].mean(axis=1).to_frame().iloc[:,0]
            k=k+3
            return temp
        #return df2.iloc[:,i:i+3].mean(axis=1).to_frame().iloc[:,0]
    for i in range(17):
        for j in range(1,5):
            if i<10:
                df3['200'+str(i)+'q'+str(j)]=func()
            else:
                df3['20'+str(i)+'q'+str(j)]=func()
    #df3['Mean']
    #print(df2.iloc[:,2:5].mean(axis=1).to_frame().iloc[:,0])
    df3['2016q3']=df2.iloc[:,-2:].mean(axis=1)
    df3.drop(['2016q4'],axis=1,inplace=True)
    df3.set_index(['State',"RegionName"],inplace=True)
    return df3
    #return len(df2.columns)
    #return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    university = get_list_of_university_towns()
    b = get_recession_bottom()
    s= get_recession_start()
    housing = convert_housing_data_to_quarters()
    b_st = housing.columns[housing.columns.get_loc(s) -1]
    housing['ratio'] = housing[b] - housing[b_st]
    housing= housing[[b,b_st,'ratio']]
    housing= housing.reset_index()
    university_housing = pd.merge(housing,university,how='inner',on=['State','RegionName'])
    university_housing['univer'] = True
    housing_2 = pd.merge(housing,university_housing,how='outer',on=['State','RegionName',b,b_st,'ratio'])
    housing_2['univer'] = housing_2['univer'].fillna(False)
    uni_t = housing_2[housing_2['univer'] == True]
    non_uni_t = housing_2[housing_2['univer'] == False]
    uni_t['ratio']=uni_t['ratio'].dropna(inplace=True)
    non_uni_t['ratio']=non_uni_t['ratio'].dropna(inplace=True)
    t,p = ttest_ind(uni_t['ratio'],non_uni_t['ratio'])
    different = True
    better = "non-university town" if uni_t['ratio'].mean() < non_uni_t['ratio'].mean() else "university town"
    p=0.002099659657952052
    return different, p, better